In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# create a model class

# inherit MODULE
class Model(nn.Module):
    
    # method 1
    def __init__(self, in_features = 4, h1 = 8, h2 = 9, out_features = 3):
        # inherit (call the nn.Module init method)
        super().__init__()
        
        # Input layer (4-features) --> h1 N --> h2 N --> output (3-classes)
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)
            
    # method 2
    def __forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        
        return x
        

In [ ]:
t